In [33]:
from pathlib import Path
import re
import queue
from itertools import combinations,chain
import matplotlib.pyplot as plt
import networkx as nx
G = nx.Graph()
file_path = "C:/Users/jinxi/OneDrive - Chalmers/courses/2022_Advent_of_Code/"
file_name = "example.txt"
file_name = "day16_input_data.txt"
with open(Path(file_path + file_name)) as f:
    data = f.read().rstrip().split("\n")
#print(data)

In [34]:
for line in data:
    #print(line)
    valves = re.findall('[A-Z][A-Z]', line)
    rate = int(re.findall('\d+', line)[0])
    start = valves[0]
    connections = valves[1:]
    G.add_node(start, rate = rate, status="closed", dist = {})
    G.add_edges_from([(start,node) for node in connections])
    #break
# plt.figure(figsize=(9,6))
# pos1 = nx.spring_layout(G)
# pos2 = {k: (x,y- 0.06)  for k, (x,y) in pos1.items()}

# nx.draw_networkx(G, with_labels=True, node_color="pink",node_size=1000,pos = pos1) 
# #nx.draw_networkx_labels(G, labels = rate_table, pos = pos2, font_color="blue")

# plt.show()

In [35]:
rate_table = nx.get_node_attributes(G, "rate")
valve_lst = [] ##collect all valves that have rate>0 and also "AA".
for node,rate in rate_table.items():
    #print(node,rate)
    if rate > 0 or node == "AA":
        valve_lst.append(node)
print(valve_lst)

distance = {} ##calculate distance from one node to another
for node1, node2 in combinations(valve_lst, 2):
    distance [node1,node2] = distance [node2,node1] = len(nx.shortest_path(G, source = node1, target = node2)) - 1
#print(distance)

['NV', 'NU', 'PS', 'FX', 'AA', 'DG', 'PH', 'KZ', 'DO', 'EJ', 'YA', 'RG', 'XK', 'UX', 'NO', 'JM']


In [36]:
def move_to_next (current_node,next_node,remaining_time,current_pressure):
    # print("\n===============")
    # print("current_node", current_node)
    # print("next node", next_node)
    
    remaining_time -= (distance[current_node,next_node] + 1)
    current_pressure += remaining_time * G.nodes[next_node]["rate"]
    #current_node = next_node

    #print("distance between",current_node,"and",next_node,"is" ,distance[current_node,next_node])
    # print("pressure", current_pressure)
            
    return remaining_time,current_pressure

In [37]:

def func_max_pressure (unopen_nodes,current_node,remaining_time,current_pressure,max_pressure,pathway):
    q = queue.Queue()
    q.put((current_node,unopen_nodes,remaining_time,current_pressure,pathway))
    #count = 0
    while not q.empty():
        current_node,unopen_nodes,remaining_time,current_pressure,pathway = q.get()
        #print("\n===============")
        #print("count",count)
        # print("current_node", current_node)
        #print("pathway", pathway)
        
        # print("unopen_nodes", unopen_nodes)
        # print("remaining_time", remaining_time)
        # print("current_pressure", current_pressure)
        #unopen_nodes.copy()
        for next_node in unopen_nodes:
            #print("\ncurrent node", next_node)
            path = pathway + "|" + next_node

            if remaining_time < (distance[current_node,next_node] + 1):
                
                #print("====break====")
                continue
            else:
                time_afterwards,pressure_afterwards = move_to_next (current_node,next_node,remaining_time,current_pressure)
                still_unopen = [n for n in unopen_nodes if n != next_node]
                if pressure_afterwards > max_pressure:
                    max_pressure = pressure_afterwards
                    opt_path = path ##optimal path
                if still_unopen:  #not empty list
                # #     continue
                    q.put((next_node,still_unopen,time_afterwards,pressure_afterwards,path))
                
                # print("time_afterwards", time_afterwards)
                # print("pressure_afterwards", pressure_afterwards)
        #count += 1       
    return max_pressure, opt_path

#print("count",count)

In [38]:
start_node = "AA"
remaining_time = 26
current_pressure = 0
max_total = 0

unopen_nodes = set(v for v in valve_lst if v != "AA")
#To reduce the search space, I guess that at least 6 valves will be open by human (or an elephant).
#if min_nr =1, the running time will increase by 10 min.
min_nr = 6 #mininal number of valves that will be open by human or elephant. 
unopen_nodes_combo = chain.from_iterable(combinations(unopen_nodes, r) for r in range(min_nr,int(len(unopen_nodes)/2)+1))

for unopen_nodes_h in unopen_nodes_combo: ##_h: human;
    unopen_nodes_e = unopen_nodes - set(unopen_nodes_h) ##_e:elephant
    
    #print("=====\nhuman nodes",unopen_nodes_h)
    #print("elephant nodes",unopen_nodes_e)

    total_pressure_h, path_h = func_max_pressure(unopen_nodes_h,start_node,remaining_time,current_pressure,max_pressure=0,pathway="AA")
    total_pressure_e, path_e = func_max_pressure(unopen_nodes_e,start_node,remaining_time,current_pressure,max_pressure=0,pathway="AA")
    total = total_pressure_h + total_pressure_e
    if total > max_total:
        max_total = total
        opt_path_h = path_h
        opt_path_e = path_e
    #print("===== The for path human is ",path_h)
    #print("===== The for elepant human is ",path_e)
    #print("===== The total presuure is ", total_pressure_h,"+",total_pressure_e,"=",total )
print("=====\nmax total pressure is ",max_total)
print("The optimal pathway for human is ",opt_path_h)
print("The optimal pathway for elephant is ",opt_path_e)

=====
max total pressure is  2594
The optimal pathway for human is  AA|NU|YA|EJ|XK|RG
The optimal pathway for elephant is  AA|NV|PS|FX|JM|KZ|UX|DO|PH
